In [1]:
import json
import random
import pandas as pd
from pathlib import Path
from tqdm.auto import tqdm
from openai import OpenAI
from react_agent import ReactInferenceAgent

In [2]:
class SingleModelOpenAIClient:
    def __init__(self, client, model):
        self._client = client
        self._model = model

    @property
    def chat(self):
        return self

    @property
    def completions(self):
        return self

    def create(self, **kwargs):
        return self._client.chat.completions.create(
            model=self._model,
            **kwargs,
        )

In [3]:
client = SingleModelOpenAIClient(
    client=OpenAI(
        api_key='EMPTY',
        base_url='http://10.239.2.27:18000/v1'
    ),
    model='Qwen3-8B'
)
agent = ReactInferenceAgent(client)

In [4]:
df = pd.read_json('validation.jsonl', lines=True)
df = df.sample(3)
# df = df.sample(3, random_state=42)
df

,question,answer
396,"Based on the provided specifications, select t...",FT_OTA
970,"Based on the provided specifications, select t...",FT_OTA
999,"Based on the provided specifications, select t...",FT_OTA


In [5]:
for i in tqdm(df.index):
    try:
        question = df.loc[i, 'question']
        answer = df.loc[i, 'answer']
        prediction, messages = agent.run(question)
        messages = json.dumps(messages, ensure_ascii=False)
        df.loc[i, 'prediction'] = prediction
        df.loc[i, 'messages'] = messages
    except:
        pass

  0%|          | 0/3 [00:00<?, ?it/s]

In [6]:
# df.to_json('Qwen3-8B.jsonl', orient='records', force_ascii=False, lines=True)

In [7]:
df

,question,answer,prediction,messages
396,"Based on the provided specifications, select t...",FT_OTA,FD_CAS_OTA,"[{""role"": ""system"", ""content"": ""You are a help..."
970,"Based on the provided specifications, select t...",FT_OTA,CM_OTA,"[{""role"": ""system"", ""content"": ""You are a help..."
999,"Based on the provided specifications, select t...",FT_OTA,CM_OTA,"[{""role"": ""system"", ""content"": ""You are a help..."
